# Computational Theory Problems

# Problem 1: Binary Words and Operations

In [ ]:
def Parity(x, y, z):
    """
    Calculate the Parity function for SHA-256.
    
    As defined in Section 4.1.2 (equation 4.3) of FIPS 180-4,
    this function returns the bitwise XOR of three 32-bit words.
    
    The formula is: Parity(x, y, z) = x ⊕ y ⊕ z
    
    Parameters
    ----------
    x : int or numpy.uint32
        First 32-bit word
    y : int or numpy.uint32
        Second 32-bit word
    z : int or numpy.uint32
        Third 32-bit word
        
    Returns
    -------
    numpy.uint32
        The bitwise XOR of x, y, and z
        
    References
    ----------
    FIPS 180-4, Section 4.1.2, page 10
    https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf
    """
    # Ensure all inputs are treated as 32-bit unsigned integers.
    # This prevents overflow and ensures compatibility with the standard.
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    # Perform bitwise XOR operation.
    # The ^ operator in Python performs bitwise XOR.
    # See: https://docs.python.org/3/reference/expressions.html#binary-bitwise-operations
    return x ^ y ^ z

# Problem 2: Fractional Parts of Cube Roots

# Problem 3: Padding

# Problem 4: Hashes

# Problem 5: Passwords

# End